<h1 align="center">Final Solution</h1>

In [1]:
import os
import pandas as pd
import random
from copy import deepcopy
data_path = os.path.join(os.getcwd(), 'Data', 'distance_matrix_official.csv')
data_matrix_df = pd.read_csv(data_path, index_col=0)
data_matrix_np = data_matrix_df.to_numpy()
data_matrix_np.shape

from Genetic_algorithm.fitness import ResourceFitness
from Genetic_algorithm.genome import Genome
from Genetic_algorithm.solution_rd import SolutionRD
from Genetic_algorithm.mutations import logistic_mutation, social_mutation
from Genetic_algorithm.crossovers import full_crossover
from Genetic_algorithm.selection_algorithms import tournament_selection
random.seed(0)

my_fitness = ResourceFitness(data_matrix_np)

first_solution = SolutionRD(my_fitness, Genome, [logistic_mutation, social_mutation], [full_crossover, full_crossover])
second_solution = SolutionRD(my_fitness, Genome, [logistic_mutation, social_mutation], [full_crossover, full_crossover])

In [2]:
import tqdm
def genetic_algorithm(
    gen_count: int,
    selection_algorithm: list[callable],
    mutation_algorithms: list[callable],
    crossover_algorithms: list[callable],
    fitness_instance: ResourceFitness,
    Genome_class: Genome,
    maximization: bool = True,
    xo_prob: float = 0.9,
    mut_prob: float = 0.2,
    social_mutation_prob: float = 0.2,
    social_crossover_prob: float = 0.2,
    population_size: int = 10,
    elitism: bool = True,
    save_logs: bool = False,
):
    """
    Genetic Algorithm for solving optimization problems.
    This function implements a genetic algorithm that evolves a population of solutions
    over a specified number of generations. It uses selection, mutation, and crossover
    algorithms to create new solutions and improve the population's fitness.
    The algorithm can be configured with various parameters, including the number of
    generations, selection method, mutation and crossover probabilities, and whether
    to use elitism.

    Args:
        gen_count (int): number of generations to evolve.
        selection_algorithm (list[callable]): selection algorithm to use.
        mutation_algorithms (list[callable]): choose from Genetic_algotithm.mutations
        crossover_algorithms (list[callable]): choose from Genetic_algorithm.crossovers
        fitness_instance (ResourceFitness): instancee of the fitness class (it's important to instanciate it before as it holds the data matrix)
        Genome_class (Genome): class of the genome to be used (will be instanciated in the function)
        maximization (bool, optional): _description_. Defaults to True.
        xo_prob (float, optional): _description_. Defaults to 0.9.
        mut_prob (float, optional): _description_. Defaults to 0.2.
        social_mutation_prob (float, optional): _description_. Defaults to 0.2.
        social_crossover_prob (float, optional): _description_. Defaults to 0.2.
        elitism (bool, optional): _description_. Defaults to True.
        verbose (bool, optional): _description_. Defaults to False.
        
    """
    
    
    
    first_solution = SolutionRD(fitness_instance, Genome_class, mutation_algorithms, crossover_algorithms)
    
    first_solution.prob_social_mutation = social_mutation_prob
    first_solution.prob_social_crossover = social_crossover_prob
    first_solution.initial_population = population_size
    
    GA_population = [random_solution for random_solution in first_solution]
    
    if save_logs:
        logs = []
    for generation in tqdm.tqdm(range(gen_count), desc="Runnig genetic alogrithm", unit=" generation"):
        fitness_instance.number_of_calls = 0
            
        # Selection
        selected_individuals = [selection_algorithm(GA_population, maximization) for _ in range(population_size)]
        

        # Crossover
        new_population = []
        for i in range(0, population_size):
            for j in range(i+1, population_size):
                if random.random() < xo_prob:
                    parent1 = selected_individuals[i]
                    parent2 = selected_individuals[j]
                    child1, child2 = parent1 @ parent2
                    new_population.append(child1)
                    new_population.append(child2)
        
        if not new_population:
            new_population = selected_individuals.copy()
                   
        
        # Mutation
        for individual in new_population:
            if random.random() < mut_prob:
                individual.mutation()
        
        # Elitism
        if elitism:
            best_individual = min(new_population) if not maximization else max(new_population)
            new_population[0] = best_individual
        
        
        GA_population = new_population
        if save_logs:
            
            n_fitness_calls = fitness_instance.number_of_calls
            best_individual_of_generation = min(GA_population) if not maximization else max(GA_population)
            best_logistic_fitness = fitness_instance._calculate_logistic_fitness(best_individual_of_generation.genome)
            best_social_fitness = fitness_instance._calculate_social_fitness(best_individual_of_generation.genome)
            row = [n_fitness_calls, float(best_individual_of_generation), best_logistic_fitness, best_social_fitness]

            logs.append(row)

            # logs_df = pd.DataFrame(logs, columns=['n_fitness_calls', 'best_individual_fitness', 'best_logistic_fitness', 'best_social_fitness'])
            # logs_df['cumulative_calls'] = logs_df['n_fitness_calls'].cumsum()
            
            # add here the savin of the logs
            #logs_df.to_csv('logs.csv', index=False)
            
       
    best_individual = min(GA_population) if not maximization else max(GA_population)
    if save_logs:
        return best_individual, logs
    return best_individual

# use
best_solution, logs = genetic_algorithm(
    gen_count=100,
    selection_algorithm=tournament_selection,
    mutation_algorithms=[logistic_mutation, social_mutation],
    crossover_algorithms=[full_crossover, full_crossover],
    fitness_instance=my_fitness,
    Genome_class=Genome,
    maximization=True,
    xo_prob=0.8,
    mut_prob=0.2,
    social_mutation_prob=0.5,
    social_crossover_prob=0.5,
    population_size=10,
    save_logs= True,
    elitism=True
)
print("Best Solution Genome:", best_solution.genome)
print("Best Solution Fitness:", float(best_solution))

Runnig genetic alogrithm: 100%|██████████| 100/100 [00:19<00:00,  5.07 generation/s]

Best Solution Genome: 01122-104-1053198276-189315067-142-1276310589-14-1
Best Solution Fitness: 4.46102629110796


In [3]:
print("Best Solution House Assignments:", best_solution.genome.house_assignments)
print("Best Solution Course Assignments:", best_solution.genome.course_assignments)

Best Solution House Assignments: [ 0  1  1  2  2 -1  0]
Best Solution Course Assignments: [ 4 -1  0  5  3  1  9  8  2  7  6 -1  8  9  3  1  5  0  6  7 -1  4  2 -1
  2  7  6  3  1  0  5  8  9 -1  4 -1]
